<a href="https://colab.research.google.com/github/vishnubob/pixace/blob/main/examples/pixace_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pixace

In [ ]:
%load_ext tensorboard

# download and install the latest version of pixace
!sudo pip install -qqq -U \
  https://github.com/vishnubob/pixace/archive/main.zip
  
import pixace

In [ ]:
# download the animalfaces model

pixace.download_model(model_name="animalfaces")

In [ ]:
# generate a new animal face from scratch

predictor = pixace.get_predictor(model_name="animalfaces")
predictor.predict()

In [ ]:
# use two images from the web as image prompts

url_1 = "https://github.com/vishnubob/pixace/raw/main/examples/prompts/prompt_1.jpg"
url_2 = "https://github.com/vishnubob/pixace/raw/main/examples/prompts/prompt_2.jpg"
prompts = [url_1, url_2]
predictor.predict(prompts=prompts, cut=512)

In [ ]:
# download emoji image set
# split images into training sets and validation sets

import os
import io
import shutil
import random

from zipfile import ZipFile
import requests

def download_and_extract(url, dest="data"):
  resp = requests.get(url)
  fh = io.BytesIO(resp.content)
  fh.seek(0)
  with ZipFile(fh) as arc:
    arc.extractall(dest)

def split_data(path=None, dest=None, ratio=0.05):
  assert ratio > 0 and ratio < 1
  files = os.listdir(path)
  random.shuffle(files)
  t_cnt = int(round(len(files) * (1 - ratio)))
  t_set = files[:t_cnt]
  v_set = files[t_cnt:]
  parts = (("train", v_set), ("val", t_set))
  
  new_paths = {}
  for (name, dataset) in parts:
    out = os.path.join(dest, name)
    new_paths[name] = out
    if os.path.isdir(out):
      shutil.rmtree(out)
    os.makedirs(out, exist_ok=True)
    for src in dataset:
      tgt = os.path.join(out, os.path.split(src)[-1])
      os.symlink(src, tgt)
  msg = f"Split {len(files)} images: {len(t_set)} training, {len(v_set)} validation"
  print(msg)
  return new_paths


url = "https://github.com/googlefonts/noto-emoji/archive/master.zip"
weight_dir = "model-weights"
out = "emoji-data"
images = f"{out}/noto-emoji-master/png/128"

os.makedirs(weight_dir, exist_ok=True)
download_and_extract(url, out)
dataset_paths = split_data(path=images, dest=out)

In [ ]:
# train a new model on the emoji image dataset
# this training run will require ~20 minutes to run
# we will dial down the image resolution to 16x16 and reduce the
# bitdepth of color to HSV 433
#
# there is a tensorboard below.  after a few minutes of training
# you can hit the refresh button on the top right of the tensorboard
# visualized samples from training are available by
# clicking on the tensorboard images tab 

%tensorboard --logdir model-weights

trainer = pixace.get_trainer(
    model_name="emoji_16_433",
    image_size=16,
    bitdepth=(4, 3, 3)
)
trainer.train(
    steps_per_epoch=200,
    images=dataset_paths["train"],
    val_images=dataset_paths["val"],
    batch_size=64,
    n_epochs=5
  )

In [ ]:
# use your new emoji model to generate new emojis
#
# Note: we haven't given the model enough time to converge, so
# don't expect these predictions to look amazing
# however, you should see hints of emoji structure and color
#
# the trainer above is smart enough to pick up where it left off, making it
# easy to continue to train your model.  try it! 

predictor = pixace.get_predictor(model_name="emoji_16_433", image_size=16, bitdepth=(4,3,3))
predictor.predict(batch_size=4, temperature=(1, 1, 1, 1))